Customer data (age, account_balance, num_products, tenure) to predict if they'll leave (binary classification).


Data is Random


Write code to:
1. Train the model for 50 epochs
2. Store the training loss history
3. Plot the training loss curve over epochs
4. Print both training and test accuracy

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

x = torch.rand(10000, 4)
y = torch.randint(low=0, high=2, size=(10000,)).float()


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

x_test = x_test.to('mps')
x_train = x_train.to('mps')
y_train = y_train.to('mps')
y_test = y_test.to('mps')

train_data = TensorDataset(x_train, y_train)
test_data = TensorDataset(x_test, y_test)

train_data = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = DataLoader(test_data, batch_size=64, shuffle=True)

class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.f1 = nn.Linear(4, 128)
        self.f2 = nn.Linear(128, 32)
        self.output = nn.Linear(32, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.f1(x))
        x = self.relu(self.f2(x))
        x = self.sigmoid(self.output(x))

        return x


model = ChurnModel()
model = model.to('mps')

epochs = 50
crit = nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.02)
loss_array = []
for i in range(epochs):
    model.train()
    t_loss = 0
    for xi, yi in train_data:
        optim.zero_grad()
        pred = model(xi)
        loss = crit(pred, yi)
        t_loss += loss
        loss.backward()
        optim.step()
    loss_array.append(t_loss.item()/len(train_data))

model.eval()
loss = 0
with torch.no_grad():
    for xi, yi in test_data:
        pred = model(xi)
        loss += crit(pred, yi).item()

print(loss/len(test_data))
print(sum(loss_array))


0.6957074422389269
34.68434075927734
